In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio


2025-02-22 15:49:13.094633: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 15:49:13.096978: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 15:49:13.104548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-22 15:49:13.119592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-22 15:49:13.119640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 15:49:13.129845: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
import utilities as util

In [3]:
# Create a dataset from a csv file

csv_file = "dataClassified/data.csv"

# csv is in format: filename, label

prepend_path = "dataClassified/"



In [4]:
# Load datasets from util.Dataset

dataset_train = util.CustomDataset(csv_file, prepend_path, "train")
dataset_val = util.CustomDataset(csv_file, prepend_path, "val")
dataset_test = util.CustomDataset(csv_file, prepend_path, "test")

2025-02-22 15:49:16.423495: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-22 15:49:16.428595: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)


In [6]:
import finetuning_utils as ftu

In [7]:
embeds, labels = ftu.preprocess_dataset(dataset_train, yamnet_model)


2025-02-22 15:49:39.615060: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
embeds_val, labels_val = ftu.preprocess_dataset(dataset_val, yamnet_model)

2025-02-22 15:49:45.956608: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
embeds_test, labels_test = ftu.preprocess_dataset(dataset_test, yamnet_model)

2025-02-22 15:49:50.859715: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
# Now, we will define a new Classifier model that will take the embeddings as input and output the class probabilities

from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.Sequential([
    layers.Input(shape=(1024,)),
    
    # First Dense Layer with L2 Regularization and Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    
    # Second Dense Layer with L2 Regularization and Dropout
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),

    # Output Layer
    layers.Dense(util.num_classes, activation='softmax')
])



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [21]:
embeds.shape, labels.shape

(TensorShape([20816, 1024]), TensorShape([20816]))

In [23]:
# Now we will train the model

history = model.fit(embeds, labels, epochs=50, validation_data=(embeds_val, labels_val)) #callbacks=[early_stop])



Epoch 1/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9350 - loss: 0.2287 - val_accuracy: 0.9002 - val_loss: 0.3366
Epoch 2/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9377 - loss: 0.2221 - val_accuracy: 0.8833 - val_loss: 0.3615
Epoch 3/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9396 - loss: 0.2209 - val_accuracy: 0.8889 - val_loss: 0.3686
Epoch 4/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9444 - loss: 0.2116 - val_accuracy: 0.8858 - val_loss: 0.3673
Epoch 5/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9445 - loss: 0.2113 - val_accuracy: 0.8953 - val_loss: 0.3536
Epoch 6/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9435 - loss: 0.2106 - val_accuracy: 0.8671 - val_loss: 0.4540
Epoch 7/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9427 - loss: 0.2110 - val_accuracy: 0.8956 - val_loss: 0.3568
Epoch 8/50
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9477 - loss: 0.2018 - val_accuracy: 0.

In [12]:
# Check the model performance on the test set

test_loss, test_acc = model.evaluate(embeds_test, labels_test, verbose=2)
print('\nTest accuracy:', test_acc)

132/132 - 0s - 899us/step - accuracy: 0.8934 - loss: 0.3596

Test accuracy: 0.8934406638145447
